## Category X Analysis Workbook Template
**Written by: INSERT NAME**

In [1]:
%pip install -r requirements.txt --upgrade

  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from p2utilityfunctions import solvercomplete, nutrition
#for documentation on any function run "function?", ex nutrition?
import numpy as np
import pandas as pd
from eep153_tools.sheets import read_sheets

### Part 1: Setting constraints and general variables

In [3]:
age = 25 # change this depending on prison average
sex = "F" #change this as well; M for Male and F for Female
activity_level = "Moderately Active" #change this; can be Moderately Active, Sedentary, or Active

In [ ]:
# time to make minimizing and maximizing constraints
bmin, bmax = nutrition(age, sex, activity_level)
print(f"Current minimizing constraints are: \n \n {bmin}")
print(f"Current maximum constraints are: \n \n {bmax}")

#### A Note On Case Specific Constraint Editing
Feel free to add in other maximizing constraints if you think they are relevant. Just make sure to justify them

### Part 2: Determining recipe and nutrition set

**Nutrition set is what constrains the options, since we are using a left join (or merge) on nutrients and recipes. The Nutrition dataset provides nutritional values and the recipe set provides weights**

In [ ]:
data_url = "https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing"
recipes = read_sheets(data_url, sheet="recipes")
nutrients = read_sheets(data_url, sheet="nutrients")

In [ ]:
nutrients.head()

**Now, drop any foods within ingredients that you want in order to constrain to the specific area of interest (ex. particular population)**

Below is a search function. Feel free to use it to either view particular foods within the dataset or set cut = True if you want to simply cut out all food options in the final result that have the search term in it.

In [ ]:
def nutrient_search(search_term, nutrients, cut = False):
    """
    Filters the nutrients DataFrame based on the presence or absence of a search term in the 'Ingredient description' column.

    When 'cut' is True, the function returns only the rows where the 'Ingredient description' contains the given search term.
    When 'cut' is False, it returns only the rows where the 'Ingredient description' does NOT contain the search term.

    Parameters:
        search_term (str): The term to search for in the 'Ingredient description' column.
        nutrients (pd.DataFrame): The DataFrame containing nutrient data with an 'Ingredient description' column.
        cut (bool, optional): Determines the filtering mode. If True, select rows containing the search term.
                              If False (default), select rows that do not contain the search term.

    Returns:
        pd.DataFrame: The filtered DataFrame based on the specified condition.
    """
    if cut:
        return nutrients[~nutrients['Ingredient description'].str.contains(search_term)]
    else:
        return nutrients[nutrients['Ingredient description'].str.contains(search_term)]

In [ ]:
#example use
nutrient_search('Butter', nutrients).head()

In [ ]:
## Running the optimization

In [ ]:
solvercomplete(sex, age,recipes, nutrients, bmin, bmax)

In [68]:
solvercomplete?

Signature:
solvercomplete(
    sex,
    age,
    recipes,
    bmin,
    bmax,
    data_url='https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing',
)
Docstring:
Constructs and solves a diet optimization problem based on recipes, nutritional data, and prices.

Before running this function, ensure that you have assigned the `recipes` dataframe 
using:
    recipes = read_sheets(data_url, sheet="recipes")
or a constrained recipes sheet (e.g., for prisons) so that `recipes` contains the desired data.

This function performs the following steps:
  1. Preprocesses the `recipes` dataframe by:
     - Formatting food codes (columns "parent_foodcode" and "ingred_code") using the `format_id` function.
     - Renaming "parent_desc" to "recipe".
     - Normalizing ingredient weights within each recipe to percentage terms.
  2. Reads and preprocesses the `nutrients` data from the provided data URL, 
     formatting the "ingred_code" column.
  3. Merges t